In [1]:
!pip install mamba_ssm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.0 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  

## Added bottleneck layer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("nvidia/MambaVision-S-1K", trust_remote_code=True)
model.cuda()  # Move model to GPU

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/36.4k [00:00<?, ?B/s]

configuration_mambavision.py:   0%|          | 0.00/625 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nvidia/MambaVision-S-1K:
- configuration_mambavision.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_mambavision.py:   0%|          | 0.00/27.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nvidia/MambaVision-S-1K:
- modeling_mambavision.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


model.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

MambaVisionModelForImageClassification(
  (model): MambaVision(
    (patch_embed): PatchEmbed(
      (proj): Identity()
      (conv_down): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (4): BatchNorm2d(96, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (levels): ModuleList(
      (0): MambaVisionLayer(
        (blocks): ModuleList(
          (0): ConvBlock(
            (conv1): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act1): GELU(approximate='tanh')
            (conv2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1,

In [4]:
import torch
import torch.nn as nn
class Bottleneck(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        self.conv = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=1),
            nn.BatchNorm2d(dim),
            nn.Conv2d(dim, dim*4, kernel_size=1),
            nn.GELU(),
            nn.Conv2d(dim*4, dim, kernel_size=1)
        )

    def forward(self, x):
        original_shape = x.shape
        input_dim = x.dim()

        # Handle 3D input [B, L, C]
        if input_dim == 3:
            B, L, C = x.shape
            H = W = int(L**0.5)  # Assume square patches
            x = x.view(B, H, W, C).permute(0, 3, 1, 2)

        # Handle 4D input [B, H, W, C]
        elif input_dim == 4 and x.shape[-1] == self.dim:
            x = x.permute(0, 3, 1, 2)

        # Handle unexpected shapes
        else:
            raise ValueError(f"Unexpected input shape: {original_shape}")

        # Apply convolution
        x = self.conv(x)

        # Restore original shape
        if input_dim == 3:
            x = x.permute(0, 2, 3, 1).reshape(B, L, C)
        else:  # input_dim == 4
            x = x.permute(0, 2, 3, 1)

        return x

# Insert into levels[2]
level_2 = model.model.levels[2]
level_2.blocks.insert(3, Bottleneck(dim=384))  # Adjust index

In [ ]:
model

MambaVisionModelForImageClassification(
  (model): MambaVision(
    (patch_embed): PatchEmbed(
      (proj): Identity()
      (conv_down): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (4): BatchNorm2d(96, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (levels): ModuleList(
      (0): MambaVisionLayer(
        (blocks): ModuleList(
          (0): ConvBlock(
            (conv1): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act1): GELU(approximate='tanh')
            (conv2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1,

In [5]:
bottleneck = Bottleneck(dim=384)
dummy_input = torch.randn(1, 32, 196, 384)  # Match your error shape
output = bottleneck(dummy_input)
print(output.shape)  # Should be [1, 32, 196, 384]

torch.Size([1, 32, 196, 384])


In [23]:
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

# Define transforms
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

])


from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Load datasets
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/MINI_PRO/MINI_PRO/training_set', transform=train_transform)
test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/MINI_PRO/MINI_PRO/test_set', transform=test_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [24]:
from torch.optim import AdamW
device ='cuda'
criterion = nn.BCEWithLogitsLoss()
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
model = model.to(device)
import torch.nn as nn

# Assuming your model is instantiated as 'model'
num_features = model.model.head.in_features  # Get the input features of the current head
model.model.head = nn.Linear(num_features, 1)  # Replace with binary classification head

# Move model to device
model = model.to(device)

In [29]:
import torch
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

model.eval()

# Tracking misclassified images (store filename, predicted class, true class)
from sklearn.metrics import accuracy_score, roc_auc_score
import torch

def evaluate(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    misclassified_info = []

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)["logits"].squeeze()
            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs > 0.5).astype(int)

            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

            # Get filenames for current batch
            batch_filenames = [loader.dataset.samples[i][0] for i in
                               range(batch_idx * loader.batch_size,
                                     batch_idx * loader.batch_size + len(labels))]

            for i in range(len(labels)):
                pred_label = int(preds[i])
                true_label = int(labels[i].item())
                if pred_label != true_label:
                    filename_short = batch_filenames[i].split("/")[-1]
                    misclassified_info.append((filename_short, pred_label, true_label))

    accuracy = accuracy_score(all_labels, all_preds)
    auroc = roc_auc_score(all_labels, all_preds)

    return accuracy, auroc, misclassified_info


# -----------------------------
# Training Loop
# -----------------------------

best_test_accuracy = 0
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device).float()

        optimizer.zero_grad()
        outputs = model(images)["logits"].squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)

    # Evaluate
    test_accuracy, test_auroc, misclassified_info = evaluate(model, test_loader, device)

    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Test Accuracy: {test_accuracy:.4f} | Test AUROC: {test_auroc:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



Epoch 1/20
Train Loss: 0.0050 | Test Accuracy: 0.9842 | Test AUROC: 0.9815

Epoch 2/20
Train Loss: 0.0049 | Test Accuracy: 0.9842 | Test AUROC: 0.9836

Epoch 3/20
Train Loss: 0.0038 | Test Accuracy: 0.9810 | Test AUROC: 0.9770

Epoch 4/20
Train Loss: 0.0249 | Test Accuracy: 0.9810 | Test AUROC: 0.9791

Epoch 5/20
Train Loss: 0.0063 | Test Accuracy: 0.9842 | Test AUROC: 0.9815

Epoch 6/20
Train Loss: 0.0039 | Test Accuracy: 0.9842 | Test AUROC: 0.9836

Epoch 7/20
Train Loss: 0.0047 | Test Accuracy: 0.9842 | Test AUROC: 0.9794

Epoch 8/20
Train Loss: 0.0080 | Test Accuracy: 0.9842 | Test AUROC: 0.9857

Epoch 9/20
Train Loss: 0.0221 | Test Accuracy: 0.9810 | Test AUROC: 0.9770

Epoch 10/20
Train Loss: 0.0186 | Test Accuracy: 0.9873 | Test AUROC: 0.9882

Epoch 11/20
Train Loss: 0.0132 | Test Accuracy: 0.9778 | Test AUROC: 0.9745

Epoch 12/20
Train Loss: 0.0207 | Test Accuracy: 0.9747 | Test AUROC: 0.9700

Epoch 13/20
Train Loss: 0.0120 | Test Accuracy: 0.9810 | Test AUROC: 0.9833

Epoch 1

MambaVisionModelForImageClassification(
  (model): MambaVision(
    (patch_embed): PatchEmbed(
      (proj): Identity()
      (conv_down): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (4): BatchNorm2d(96, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (levels): ModuleList(
      (0): MambaVisionLayer(
        (blocks): ModuleList(
          (0): ConvBlock(
            (conv1): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act1): GELU(approximate='tanh')
            (conv2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1,


✅ Test Accuracy: 66.05%


In [ ]:
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.to(device)
model.eval()


UnpicklingError: Weights only load failed. In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
Please file an issue with the following so that we can make `weights_only=True` compatible with your use case: WeightsUnpickler error: Unsupported operand 241

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.

best_model.pth: data


In [ ]:
from google.colab import files
files.upload()


Saving best_model.pth to best_model.pth


In [ ]:
!file best_model.pth
!ls -lh best_model.pth


best_model.pth: Zip archive data, at least v0.0 to extract, compression method=store
-rw-r--r-- 1 root root 221M Apr 16 16:33 best_model.pth


In [ ]:
!unzip best_model.pth -d model_zip_contents


Archive:  best_model.pth
 extracting: model_zip_contents/best_model/data.pkl  
 extracting: model_zip_contents/best_model/byteorder  
 extracting: model_zip_contents/best_model/data/0  
 extracting: model_zip_contents/best_model/data/1  
 extracting: model_zip_contents/best_model/data/10  
 extracting: model_zip_contents/best_model/data/100  
 extracting: model_zip_contents/best_model/data/101  
 extracting: model_zip_contents/best_model/data/102  
 extracting: model_zip_contents/best_model/data/103  
 extracting: model_zip_contents/best_model/data/104  
 extracting: model_zip_contents/best_model/data/105  
 extracting: model_zip_contents/best_model/data/106  
 extracting: model_zip_contents/best_model/data/107  
 extracting: model_zip_contents/best_model/data/108  
 extracting: model_zip_contents/best_model/data/109  
 extracting: model_zip_contents/best_model/data/11  
 extracting: model_zip_contents/best_model/data/110  
 extracting: model_zip_contents/best_model/data/111  
 extract

In [ ]:
!!ls model_zip_contents


['best_model']

In [ ]:
!file model_zip_contents/best_model


model_zip_contents/best_model: cannot open `model_zip_contents/best_model' (No such file or directory)
